In [93]:
class Tree:
    def leaf(data):
        return Tree(data=data)
    def __repr__(self):
        if self.is_leaf():
            return "Leaf(%r)" % self.data
        else:
            return "Tree(%r) { left = %r, right = %r }" % (self.data, self.left, self.right) 
            
    def __init__(self, *, data = None, left = None, right = None):
        self.data = data
        self.left = left
        self.right = right
    
    def is_leaf(self):
        return self.left == None and self.right == None

    def children(self):
        return [x for x in [self.left, self.right] if x]

    def depth(self):
        return max([x.depth() for x in self.children()], default=0) + 1

tree = Tree(data='isSystems?')
tree.left = Tree.leaf(data='like')
tree.right = Tree.leaf(data='TakenOtherSys?')
l1 = Tree.leaf('like')
l2 = Tree.leaf('like')
l3 = Tree.leaf('nah')
l4 = Tree.leaf('nah')
l5 = Tree.leaf('like')
tree4 = Tree(data='likeOtherSys',left=l3,right=l2)
tree3 = Tree(data='morning',left=l5,right=l4)
tree2 = Tree(data='takenOtherSys?', left=tree3, right=tree4)
tree1 = Tree(data='isSystems?', left=l1, right=tree2)


In [94]:
tree1.depth()

4

In [95]:
'''In your python code, load the following dataset and add a boolean "ok" column,
where "True" means the rating is non-negative and "False" means the rating is negative'''
import pandas as pd
dataset = pd.read_csv('Рабочий стол/dataset.csv')
values = []
for i in dataset['rating']:
    if i>=0:
        values.append(True)
    else:
        values.append(False)
dataset['ok'] = values

In [96]:
dataset

,rating,easy,ai,systems,theory,morning,ok
0,2,True,True,False,True,False,True
1,2,True,True,False,True,False,True
2,2,False,True,False,False,False,True
3,2,False,False,False,True,False,True
4,2,False,True,True,False,True,True
5,1,True,True,False,False,False,True
6,1,True,True,False,True,False,True
7,1,False,True,False,True,False,True
8,0,False,False,False,False,True,True
9,0,True,False,False,True,True,True


In [176]:
#df[[feature, goal]].value_counts()    
# это мы оцениваем одну фичу или один вопрос
def single_feature_score(data, goal, feature):
    # это список всех значений из столбца с фичей,
    # где напротив этих значений в столбце окей стоит тру
    # а в Нет соответсвенно, наоборот
    # таким образом, мы делим столбец на две части. В первой(где в окей стоит тру),
    # у нас все ответы(и да, и нет), котрые привели к положительной оценке курса
    # а во второй части все ответы, которые привели к отрицательной оценке
    YES = []
    NO = []  
    for i in range(len(feature)):
        if goal[i] == True:
            YES.append(feature[i])
        if goal[i] == False:
            NO.append(feature[i])
    # далее в каждом списке считаем количество ответов НЕТ и количество ответов ДА
    # и находим каких ответов больше, то есть максимум из двух сумм
    NO_true_count = NO.count(True)
    NO_false_count = NO.count(False)
    NO = max(NO_true_count, NO_false_count)
    YES_true_count = YES.count(True)
    YES_false_count = YES.count(False)
    YES = max(YES_true_count, YES_false_count)
    # далее по логике алгоритма надо сложить максимумы из ДА и из НЕТ,
    # это и будет ответом, который мы возвращаем
    score = YES + NO
    return score

single_feature_score(dataset, dataset['ok'], dataset['systems'])


18

In [177]:
# тут мы перебираем все фичи и выбираем самую лучшую
columns = list(dataset)
score_list = []
feature_dict = {}
for i in columns:
    if i != 'ok' and i != 'rating':
        feature_score = single_feature_score(dataset, dataset['ok'], dataset[i])
        feature_dict[i] = feature_score
        score_list.append(feature_score)
        best = max(score_list)
    for k, item in feature_dict.items():
        if item == best:
            best_feature = k
print(best_feature)

systems


In [178]:
def best_feature(data, goal):
    columns = list(dataset)
    score_list = []
    feature_dict = {}
    for i in columns:
        if i != 'ok' and i != 'rating':
            feature_score = single_feature_score(dataset, goal, dataset[i])
            feature_dict[i] = feature_score
            score_list.append(feature_score)
            best = max(score_list)
            worst = min(score_list)
        for k, item in feature_dict.items():
            if item == best:
                best_feature =
            if item == worst:
                worst_feature = k
    #return best_feature, worst_feature
    return best_feature
best_feature(dataset, dataset['ok']) # 'systems'
# best_feature(dataset, dataset['ok'])[1] # 'easy'

'systems'

In [179]:
# собираем входные данные для дерева решений
columns = list(dataset)
remaining_features = []
for i in columns:
    if i != 'ok' and i != 'rating':
        remaining_features.append(i)
print(remaining_features)        

['easy', 'ai', 'systems', 'theory', 'morning']


In [180]:
import random
def DecisionTreeTrain(dataset,remaining_features):
    # находим количество тру и фолс во всём датасете
    true = []
    false = []
    for i in remaining_features:
        for value in dataset[i]:
            if value == True:
                true.append(value)
            else:
                false.append(value)
    #print(remaining_features, 'all the features')
    # determine guess value
    guess = True
    unambigous = True
    if len(true) > len(false):
        guess = True
    elif len(false) < len(true):
        guess = False
    else:
        unambigous = False        
        k = random.randint(0, 1)
        if k == 1: 
            guess = True
        else: 
            guess = False
    
    if unambigous:
        return Tree.leaf(guess)
    elif remaining_features == []:
        return Tree.leaf(guess)
    else:
        # это я нашла лучшую фичу
        current_best_feature = best_feature(dataset, dataset['ok']) 
        print(current_best_feature,'current_best_feature')
        #remaining_features.remove(current_best_feature)
        #print(remaining_features)
        #print(dataset[current_best_feature],'dataset[current_best_feature]')
        NO = dataset[dataset[current_best_feature] == False]
        #print(NO)
        YES = dataset[dataset[current_best_feature] == True]
        #print(YES)
        # print(remaining_features)
        # remove best feature from remaining feature list
        #if current_best_feature in remaining_features:
        #print(remaining_features,'remaining_features')
        remaining_features.remove(current_best_feature)
        left = DecisionTreeTrain(NO, remaining_features)
        right = DecisionTreeTrain(YES, remaining_features)
        return Tree(data = current_best_feature, left = left, right = right) 

In [181]:
DecisionTreeTrain(dataset,remaining_features)

systems current_best_feature


KeyError: 0

In [88]:
def depth():
    for i in range():
        tree = DecisionTreeTrain(dataset,remaining_features)
    return tree

In [89]:
depth(3)

TypeError: depth() takes 0 positional arguments but 1 was given